In [1]:
import sys
sys.path.append("/home/leffff/PycharmProjects/sber-scs/venv/lib/python3.10/site-packages")
import os
from joblib import load
from copy import copy
import seaborn as sns

from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, Subset
from transformers import BertForNextSentencePrediction, BertTokenizerFast, BertConfig, AutoModel, AutoTokenizer
from transformers import get_cosine_schedule_with_warmup

/home/leffff/PycharmProjects/af-bot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"ai-forever/ruBert-large"
"ai-forever/ruBert-base"
"tinkoff-ai/response-quality-classifier-base"
MODEL_NAME = "ai-forever/ruElectra-medium"

In [3]:
DEVICE = "cuda"

In [4]:
config = BertConfig.from_pretrained(MODEL_NAME)
model = BertForNextSentencePrediction(config)
model.cls.seq_relationship = nn.Linear(576, 2)
model.to(DEVICE)
print(model.config)
# model.load_state_dict(torch.load("checkpoints/2.pt"))
model.eval()

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


BertConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 576,
  "generator_size": "0.25",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 576,
  "initializer_range": 0.02,
  "intermediate_size": 2304,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 9,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 64000
}



BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 576, padding_idx=0)
      (position_embeddings): Embedding(512, 576)
      (token_type_embeddings): Embedding(2, 576)
      (LayerNorm): LayerNorm((576,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=576, out_features=576, bias=True)
              (key): Linear(in_features=576, out_features=576, bias=True)
              (value): Linear(in_features=576, out_features=576, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=576, out_features=576, bias=True)
              (LayerNorm): LayerNorm((576,), eps=1e-12,

In [5]:
# from transformers import AutoModelForSeq2SeqLM
# from peft import get_peft_model, LoraConfig, TaskType

# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
# )

# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [6]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [7]:
from datasets import load_dataset, Dataset

dataset = load_dataset("Den4ikAI/russian_dialogues", split='train').shuffle(seed=42)
dataset

Dataset({
    features: ['question', 'answer', 'relevance'],
    num_rows: 2477321
})

In [8]:
import pandas as pd

df = pd.DataFrame(dataset)
df = df.dropna()
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['question', 'answer', 'relevance', '__index_level_0__'],
    num_rows: 2476083
})

In [9]:
encoded_dataset = dataset.map(
    lambda sample: tokenizer(
        sample['question'], sample['answer'], truncation=True, padding='max_length', max_length=256
    ),
    batched=True,
)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2476083/2476083 [02:54<00:00, 14155.57 examples/s]


In [10]:
list(encoded_dataset[0].keys())

['question',
 'answer',
 'relevance',
 '__index_level_0__',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [11]:
encoded_dataset = encoded_dataset.remove_columns([
 'question',
 'answer',
 '__index_level_0__',
])

list(encoded_dataset[0].keys())

['relevance', 'input_ids', 'token_type_ids', 'attention_mask']

In [12]:
encoded_dataset.set_format(type='torch', columns=['relevance', 'input_ids', 'token_type_ids', 'attention_mask'])

In [13]:
train_inds, test_inds = train_test_split([i for i in range(len(encoded_dataset))], test_size=0.2)

train_dataloader = DataLoader(
    Subset(encoded_dataset, train_inds), 
    batch_size=76,
    shuffle=True
)

val_dataloader = DataLoader(
    Subset(encoded_dataset, test_inds), 
    batch_size=76,
    shuffle=False
)

In [14]:
from torchmetrics.classification import BinaryAUROC

ROCAUC = BinaryAUROC()

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim


def train_epoch(model, dataloader, optimizer, scheduler, criterion):
    model.train()

    all_probas = []
    all_labels = []

    total_loss = 0

    for batch in tqdm(dataloader):
        input_ids, attention_masks, token_type_ids, labels = batch["input_ids"], batch["attention_mask"], batch["token_type_ids"], batch["relevance"]
        input_ids, attention_masks, token_type_ids, labels = input_ids.to(DEVICE), attention_masks.to(DEVICE), token_type_ids.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(
            input_ids=input_ids, 
            attention_mask=attention_masks,
            token_type_ids=token_type_ids
        ).logits

        loss = criterion(output, labels)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()

        probas = output.softmax(dim=-1)
        all_probas.append(probas.detach().cpu())
        all_labels.append(labels.cpu())
        
    metrics = {
        "ROCAUC": ROCAUC(torch.cat(all_probas, dim=0)[:, 1], torch.cat(all_labels, dim=0)),
        "Loss": total_loss / len(dataloader)
    }
        
    return metrics



def val_epoch(model, dataloader, criterion):
    model.eval()
    
    all_probas = []
    all_labels = []

    total_loss = 0

    for batch in tqdm(dataloader):
        input_ids, attention_masks, token_type_ids, labels = batch["input_ids"], batch["attention_mask"], batch["token_type_ids"], batch["relevance"]
        input_ids, attention_masks, token_type_ids, labels = input_ids.to(DEVICE), attention_masks.to(DEVICE), token_type_ids.to(DEVICE), labels.to(DEVICE)

        with torch.no_grad():
            output = model(
                input_ids=input_ids, 
                attention_mask=attention_masks,
                token_type_ids=token_type_ids
            ).logits

        loss = criterion(output, labels)
        total_loss += loss.item()

        probas = output.softmax(dim=-1)
        all_probas.append(probas.detach().cpu())
        all_labels.append(labels.cpu())
        

    metrics = {
        "ROCAUC": ROCAUC(torch.cat(all_probas, dim=0)[:, 1], torch.cat(all_labels, dim=0)),
        "Loss": total_loss / len(dataloader)
    }
        
    return metrics


def train_loop(
    model, 
    epochs,
    train_dataloader,
    test_dataloader,
    optimizer,
    scheduler,
    criterion
):
    for i in range(epochs):
        train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, criterion)
        test_loss = val_epoch(model, test_dataloader, criterion)

        print(train_loss, test_loss)

        torch.save(model.state_dict(), f"electra/{i}.pt")

In [16]:
from madgrad import MADGRAD

epochs = 20
optimizer = MADGRAD([
        {"params": model.cls.parameters(), "lr": 1e-4},
        {"params": model.bert.parameters(), "lr": 5e-6},
])
total_steps = int(len(train_dataloader) * epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                    num_warmup_steps = 0, # Default value in run_glue.py
                                    num_training_steps = total_steps)

train_loop(
    model=model, 
    epochs=epochs,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=nn.CrossEntropyLoss(),
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6517/6517 [10:42<00:00, 10.14it/s]


{'ROCAUC': tensor(0.7484), 'Loss': 0.5820043456033029} {'ROCAUC': tensor(0.7690), 'Loss': 0.5696628986165735}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6517/6517 [10:45<00:00, 10.09it/s]


{'ROCAUC': tensor(0.7757), 'Loss': 0.5572885225995955} {'ROCAUC': tensor(0.7905), 'Loss': 0.54619039610904}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6517/6517 [09:34<00:00, 11.34it/s]


{'ROCAUC': tensor(0.8260), 'Loss': 0.5038450323406849} {'ROCAUC': tensor(0.8565), 'Loss': 0.4671186056466525}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6517/6517 [10:43<00:00, 10.13it/s]


{'ROCAUC': tensor(0.8759), 'Loss': 0.43368750158550873} {'ROCAUC': tensor(0.8766), 'Loss': 0.4372068587116107}


  7%|███████▉                                                                                                              | 1741/26065 [07:46<1:48:43,  3.73it/s]


KeyboardInterrupt: 